In [1]:
import matplotlib.pyplot as plt
import matplotlib

%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2


import numpy as np
#%matplotlib inline

import os
from utils import *

import os

from utils_calcium import ProcessCalcium, get_reward_triggered_psth, plot_multi_session_psth_imshow, plot_psth

Autosaving every 180 seconds


/home/cat/.conda/envs/bmitools/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [2]:
####################################################
################## LOAD DATA #######################
####################################################

#root_dir = '/media/cat/4TB/donato/andres/master_analysis/'
root_dir = '/media/cat/8TB/donato/bmi/'

animal_ids = [
    # M1 mice
    #"DON-011733",    # M1    - Processed
    #"DON-014618",    # M1    - Processed
    #'DON-014451',    # M1    - Processed
    #'DON-014382',    # M1    - Processed
    #'DON-014575',    # M1    - Processed
    #'DON-014618',    # M1    - Processed
    #'DON-015821',    # M1    - Processed
    # 'DON-017923',   # M1    - Not Processed <has extra days at the end....
     'DON-018129',   # M1    - Not Processed
     #'DON-018130',   # M1    - Not Processed

    # MAYBE MICE
    # 'DON-013392'  <-- only last day was good, all others were at chance
    # 'DON-015962'  <-- might be ok, rechedk it.

    # CA3 mice
    #"DON-012502",    # CA3    - spreadsheet is old - need reprocessing; ALSO Weird performance, may exclude
    #"DON-014266",    # CA3   - Processed
    #'DON-014371',    # CA3   - Processed
    #'DON-015801',    # CA3   - Processed
    #'DON-016658',    # CA3   - Processed
]

#
for animal_id in animal_ids:
    #       
    c = ProcessCalcium(root_dir,
                       animal_id)

    #
    c.remove_bad_cells = True
    c.self_nonmerged_rois = True
    c.use_non_merged = True

    #
    c.recompute_binarization = False  # IF TRue: must input correct params from bin notebooks
    c.dff_min = 0.05                  # min %DFF for [ca] burst to considered a spike (default 5%) overwrites percentile threshold parameter
    c.percentile_threshold = 0.999    # this is pretty fixed, we don't change it; we want [ca] bursts that are well outside the "physics-caused"noise
    c.maximum_std_of_signal = 0.25    # if std of signal is greater than this, then we have a noisy signal and we don't want to binarize it

    #
    c.save_figures = c.recompute_binarization
    
    # check if animal has been processed
    if os.path.exists(c.fname_animal):
        c = c.load_animal()
        continue
    #
    c.load_sessions()

    #
    c.save_animal()

#
print ("....DONE...")


...loading animal from disk...
....DONE...


In [3]:
########################################################
########################################################
########################################################

# adds trial information; eventually should also add bursts etc...
c.fix_spreadsheet()

#

processing session:  20231011
processing session:  20231012
processing session:  20231013
processing session:  20231014
processing session:  20231015
processing session:  20231016
processing session:  20231017
processing session:  20231018
...done...


In [4]:
########################################################
########################################################
########################################################
# load the ensemble state, trial starts, and trial ends info

# c.session_id = 8
print (c.session_ids)
c.fix_spreadsheet_missing_vals = True

#
print (c.session_types)
c.load_spreadsheet_multi()

[20231010, 20231011, 20231012, 20231013, 20231014, 20231015, 20231016, 20231017, 20231018]
['day0', 'bmi1', 'bmi2', 'bmi3', 'bmi4', 'bmi5', 'bmi6', 'bmi7', 'bmi8']


  0%|          | 0/9 [00:00<?, ?it/s]

100%|██████████| 9/9 [00:00<00:00, 63.03it/s]


In [5]:
########################################################
####### VISUALIZE SINGLE CELLS Oover threshold #########
########################################################
from utils import get_ave_maps, make_umap_pca

session_ids = np.arange(1, len(c.session_ids),1)
#session_ids = np.arange(5, len(c.session_ids),1)

#
c.thresh_zscore = 2.
c.thresh_ave = 0.2
c.use_zscore = False
c.use_ave_thresh = True
c.bin_width = 1
c.normalize_trace = False

#
c = get_ave_maps(c,
                session_ids,
                )

#################################################
############## do PCA on the sum_maps ###########
#################################################
#
print ("n_cells: ", c.n_cells)
print ("n_rewards: ", c.n_rewards)

#
make_umap_pca(c)

  0%|          | 0/8 [00:00<?, ?it/s]

100%|██████████| 8/8 [00:02<00:00,  3.42it/s]


ave_maps:  (145, 600)
clrs_maps:  (145, 4)
n_cells:  [92, 6, 1, 45, 0, 1, 0, 0]
n_rewards:  [12, 37, 39, 64, 87, 63, 101, 75]
X_in:  (145, 600)
PCA variance explained:  [0.2097228  0.11101188 0.0916502  0.06155457 0.05018188]
sum_maps_pca:  (145, 145)
20231011 , ctr:  0  k:  92
20231012 , ctr:  92  k:  6
20231013 , ctr:  98  k:  1
20231014 , ctr:  99  k:  45
20231015 , ctr:  144  k:  0
20231016 , ctr:  144  k:  1
20231017 , ctr:  145  k:  0
20231018 , ctr:  145  k:  0


In [6]:
# plot ave maps on different subplots
from utils import plot_multi_session_thresholded_cells
c.window = 5 # in secs
plot_multi_session_thresholded_cells(c)


 50%|█████     | 4/8 [00:00<00:00, 14.17it/s]


IndexError: tuple index out of range

In [ ]:
#################################################
### load the cells and plot distribution of si###
#################################################

# make color map in viridis to plot 8 lines
cmap = matplotlib.cm.get_cmap('viridis')
clrs = cmap(np.linspace(0, 0.9, 8))


#
plt.figure(figsize=(15,10))
ax1=plt.subplot(221)
plt.title("Spatial Information")
plt.legend()

ax2=plt.subplot(222)
plt.title("Z-score")
plt.legend()

ax3=plt.subplot(223)
plt.title("sparsity")
plt.legend()

ax4=plt.subplot(224)
plt.title("selectivity")
plt.legend()

#
session_ids = np.arange(1,len(c.sessions),1)

#
si_array = []
si_shuffle_array = []
zscore_array = []
ave_map_array = []
sum_map_array = []
for ctr,session_id in enumerate(session_ids):
    #
    sis = []
    zscores = []
    ave_maps = []
    sparse = []
    select = []
    for k in range(len(c.sessions[session_id].F_upphase_bin)):
        fname = os.path.join(c.root_dir,
                            c.animal_id,
                            str(c.session_ids[session_id]),
                            'spatial_info',
                            str(k)+'.npz')

        #
        d = np.load(fname, 
                    allow_pickle=True)
        si = d['si']
        zscore = d['zscore']

        #
        sis.append(si)
        zscores.append(zscore)

        #
        ave_map = d['ave_map']
        ave_maps.append(ave_map)

        # 
        sum_map = d['sum_map']
        sum_map_array.append(sum_map)

        #
        sparse.append(d['sparsity'])
        select.append(d['selectivity'])



    #
    sis = np.array(sis)
    zscores = np.array(zscores)
    ave_maps = np.array(ave_maps)
    sparse = np.array(sparse)
    idx = np.where(sparse!=None)[0]
    sparse = sparse[idx]

    select = np.array(select)
    idx = np.where(select!=None)[0]
    select = select[idx]

    si_array.append(sis)
    zscore_array.append(zscores)
    ave_map_array.append(ave_maps)


    #
    y = np.histogram(sis, bins=np.arange(0,10,0.1))
    xx = y[1][:-1]
    yy = y[0]
    # smooth yy to make it look better
    yy = np.convolve(yy, np.ones(5)/5, mode='same')
    ax1.plot(xx,yy,c=clrs[ctr], label="bmi #: " + str(session_id))

    y = np.histogram(zscores, bins=np.arange(-5,6.5,0.1))
    xx = y[1][:-1]
    yy = y[0]
    # smooth yy to make it look better
    yy = np.convolve(yy, np.ones(5)/5, mode='same')
    ax2.plot(xx,yy,c=clrs[ctr], label=str(session_id))

    #
    y = np.histogram(sparse, bins=np.arange(0,1.5,0.01))
    xx = y[1][:-1]
    yy = y[0]
    # smooth yy to make it look better
    yy = np.convolve(yy, np.ones(5)/5, mode='same')
    ax3.plot(xx,yy,c=clrs[ctr], label=str(session_id))

    y = np.histogram(select, bins=np.arange(0,15,0.1))
    xx = y[1][:-1]
    yy = y[0]
    # smooth yy to make it look better
    yy = np.convolve(yy, np.ones(5)/5, mode='same')
    ax4.plot(xx,yy,c=clrs[ctr], label=str(session_id))
#
ax1.legend()

#
plt.suptitle("Spatial Information and Z-score Distribution for animal: "+c.animal_id)

#
plt.show()


In [ ]:
# plot metrics as function of session

idxs = np.arange(400)

plt.figure()
ax1 = plt.subplot(121)
plt.title("Spatial Information vs. peak ")
#
plt.ylabel("Spatial Information")
plt.xlabel("Peak response rate")
plt.ylim(0,13)
#plt.xlim(0,1)

ax2 = plt.subplot(122)
plt.title("Zscore vs. peak ")
#
plt.ylabel("Zscore Information")
plt.xlabel("Peak response rate")
plt.ylim(-7.5,20)
#plt.xlim(0,1)

################################
# make viridis colormap over 8 values
cmap = matplotlib.cm.get_cmap('viridis')
clrs = cmap(np.linspace(0, 1, 8))

##############################
for k in range(len(si_array)):
    #
    s1 = []
    zs = []
    peaks = []
    for cell_id in range(si_array[k].shape[0]):
        si = si_array[k][cell_id]
        s1.append(si)

        z = zscore_array[k][cell_id]
        zs.append(z)

        # peak
        temp = ave_map_array[k][cell_id]
        #temp = sum_map_array[k][cell_id]
        peak = np.max(temp)
        peaks.append(peak)

        #
        #if peak>0.8:
        #    ax3.plot(temp, c=clrs[k], alpha=.5)

    #
    ax1.scatter(peaks, s1, c=clrs[k], alpha=.75,edgecolors='black')
    ax2.scatter(peaks, zs, c=clrs[k], alpha=.75,edgecolors='black')

#
plt.suptitle("SI  and Z-score vs. peak response rate for animal: "+c.animal_id)

#
plt.show()




In [ ]:
##############################################################
############### BUILD CELL BURST CLASSIFIER ##################
##############################################################

def classify_cell(data):

    # do a cumulative sum
    # compute the average activity in 4 sections: -3sec to -1sec, -1sec to 0sec, 0sec to 1sec, 1sec to 3sec

    t0 = data.shape[0]//2
    fps = 30
    times = [[-3*fps,-1*fps],
            [-1*fps,0],
            [0,1*fps],
            [1*fps,3*fps]]
    
    sums = []    
    for t in times:
        temp = data[t[0]+t0:t[1]+t0]
        sums.append(temp.mean())

    sums = np.array(sums)
    
    # find class by computing the argmax
    class_id = np.argmax(sums)

    return sums, class_id
    #

##############################################################
############### BUILD CELL BURST CLASSIFIER ##################  
##############################################################




In [ ]:

#
classes = ['pre-task','driver','reward','recovery']

#
window = 101
peak_thresh = 2.0
si_thresh = 3.0
zscore_thresh = 2
peak_min  = 0.2

#
session_id = 5

#
clrs = ['black','blue','red','green']

# use gridspec to make a 2x2 plot
fig = plt.figure(figsize=(10,10))
gs = matplotlib.gridspec.GridSpec(2, 2)

ax1 = plt.subplot(gs[0,:])
# 
ax1.set_ylabel("# of cells")
ax1.set_xlabel("Session #")
ax2 = plt.subplot(gs[1,0])
ax3 = plt.subplot(gs[1,1])

#
n_rewards = []
n_cells = []
for session_id in trange(1,len(c.sessions),1):

    #
    class_array = []

    ctr =0
    for cell_id in range(c.sessions[session_id].F_upphase_bin.shape[0]):

        fname = os.path.join(c.root_dir,
                                c.animal_id,
                                str(c.session_ids[session_id]),
                                'spatial_info',
                                str(cell_id)+'.npz')

        #
        d = np.load(fname, 
                    allow_pickle=True)
        
        #
        reward_times = d['reward_times']

        #
        si = d['si']
        zscore = d['zscore']

        #
        ave_map = d['ave_map']

        # 
        sum_map = d['sum_map']
        peak = sum_map.max()

        #
        selectivity = d['selectivity']
        sparsity = d['sparsity']

        #
        selectivity = d['selectivity']
        #
        if zscore<zscore_thresh:
            continue

        if peak < (peak_min*reward_times.shape[0]):
            continue

        #
        sums, cl = classify_cell(sum_map)

        #
        class_array.append(cl)

        #
        ctr+=1
    
    # make bar plot of how many cells in each class
    class_array = np.array(class_array)

    #
    for k in range(4):
        idx = np.where(class_array==k)[0]
        ax1.bar(k+session_id*5, len(idx),
                color=clrs[k],
                alpha=1,
                label = classes[k] if session_id==1 else "")
    
    # add xtick label for session at k+session_id*5
    ax1.set_xticks(np.arange(1,len(c.sessions),1)*5, 
                   np.arange(1,len(c.sessions),1))
    ax1.legend()
    

    #
    n_rewards.append(reward_times.shape[0])
    n_cells.append(ctr)

# plot # of cells survived triage
ax2.scatter(
    np.arange(1,len(c.sessions),1),
    #session_id, 
            n_cells, 
            c='black', 
            s=100)
ax2.set_title("Task cells")
ax2.set_xlabel("Session #")
ax2.set_ylim(bottom=0)

# plot # of rewards
ax3.scatter(
        np.arange(1,len(c.sessions),1),
            n_rewards,
            s=100,
            c='blue')
ax3.set_title("of rewards")
ax3.set_xlabel("Session #")
ax3.set_ylim(bottom=0)



#
plt.suptitle("Cell classification for animal: "+c.animal_id)

plt.show()


In [ ]:
##############################################################
############# EXAMPLE OF SINGLE CELL BURSTS ##################
##############################################################
if False:      
    cell_id = 0

    #
    d = np.load('/media/cat/8TB/donato/bmi/DON-014618/20230505/spatial_info/'+str(cell_id)+'.npz', allow_pickle=True)
    rate_map = d['rate_map']

    # make a time vector that repeats every 600 time points
    n_rewards = int(rate_map.shape[0]/(30.*20.))
    print ("n_rewards: ", n_rewards)
    t = np.arange(rate_map.shape[0])/30.

    #
    plt.figure()
    plt.plot(t,d['rate_map'])

    # relabel xticks to repeat every 600 time points
    labels = np.arange(0,n_rewards+1,1)
    labels_old = np.linspace(t[0],t[-1], len(labels))
    plt.xticks(labels_old,
            labels)

    # shade background every other labels index
    for k in range(len(labels)):
        if k%2==0:
            try:
                plt.axvspan(labels_old[k], labels_old[k+1], color='grey', alpha=.2)
            except:
                pass

    # plot a vertical dashed red line every 600 time points starting at 300
    for k in range(1,len(labels)):
        plt.plot([labels_old[k]-10, labels_old[k]-10], 
                [0,1], '--', c='red')

    #
    plt.ylim(0,1)
    plt.xlim(0,labels_old[-1])

    #
    plt.xlabel("Reward #")
    plt.ylabel("UPphase")
    plt.title("Cell: "+str(cell_id))

    #
    plt.show()


In [ ]:
#
def get_si3(rate_map,
            time_map):
    
    #
    duration = np.ma.sum(time_map)
    position_PDF = time_map / (duration + np.spacing(1))

    mean_rate = np.ma.sum(rate_map * position_PDF)

    log_argument = rate_map / (mean_rate+0)
    log_argument[log_argument < 1] = 1

    #
    inf_rate = np.ma.sum(position_PDF * rate_map * np.ma.log2(log_argument))
    inf_content = inf_rate / (mean_rate+0)

    return inf_rate, inf_content



import numpy as np

def compute_skaggs(spatial_locations,
                   event_counts):
    # Sample data: List of spatial locations and corresponding event counts
    #spatial_locations = [1, 2, 3, 4, 5]
    #event_counts = [10, 20, 5, 15, 30]

    # Calculate the total number of events
    total_events = sum(event_counts)

    # Initialize spatial information (Is)
    spatial_information = 0.0

    # Calculate the spatial information using the Skaggs formula
    si_array = []
    for count in event_counts:
        probability = (count) / (total_events)

        temp = -probability * np.log(probability)
        si_array.append(temp)
        spatial_information = spatial_information + temp

    #print ("intermim si: ", spatial_information)

    # Calculate the final spatial information value
    spatial_information = np.log(len(spatial_locations)) - spatial_information

    #print(f"Spatial Information (Is): {spatial_information:.2f}")

    return spatial_information, si_array

#
def get_si4(rate_map,
            time_map,
            verbose=False):


    #
    duration = np.ma.sum(time_map)
    if verbose:
        print ("duration: ", duration)

    #
    position_PDF = time_map / (duration + np.spacing(1))
    if verbose:
        print ("position_PDF: ", position_PDF)  

    #
    mean_rate = np.ma.sum(rate_map * position_PDF)
    if verbose:
        print ("mean_rate: ", mean_rate)

    #
    log_argument = rate_map / (mean_rate)
    if verbose:
        print ("log_argument: ", log_argument)

    #
    log_argument[log_argument < 1] = 1
    if verbose:
        print ("log_argument: ", log_argument)

    #
    inf_rate = np.ma.sum(position_PDF * rate_map * np.ma.log2(log_argument))
    inf_content = inf_rate / (mean_rate)
    if verbose:
        print ("inf_rate: ", inf_rate)
        print ("inf_content: ", inf_content)

    return inf_rate, inf_content

############################################################
######################### Load Data ########################
############################################################
session_id = 7
bin_width = None
verbose = False

#
sum_maps = []
for cell_id in range(416):
    fname = os.path.join(c.root_dir,
                        c.animal_id,
                        str(c.session_ids[session_id]),
                        'spatial_info',
                        str(cell_id)+'.npz')
    #
    d = np.load(fname, allow_pickle=True)
    reward_times = d['reward_times']
    sum_map = d['sum_map']
    sum_maps.append(sum_map)


# loop over each session
plt.figure()
ctr=0

# loop over each cell 
idx_cells = np.random.choice(np.arange(len(sum_maps)), 25, replace=False)

for cell_id in np.arange(len(sum_maps)):
#idx_cells = [167, 194, 60, 35]
#for cell_id in idx_cells:
    #
    ax = plt.subplot(5,5,ctr+1)       
    
    # grab data
    rate_map = sum_maps[cell_id]

    # scale by # of rewards
    time_map = np.ones(rate_map.shape[0])*reward_times.shape[0]

    # bin rate_map and time_map into bin_width bins by summing in each bin
    if bin_width is not None:
        rate_map = np.mean(rate_map.reshape(-1, bin_width), axis=1)
        time_map = np.mean(time_map.reshape(-1, bin_width), axis=1)
        
        #
        if verbose:
            print ("rate_map: ", rate_map.shape, rate_map[:10])
            print ("time_map: ", time_map.shape, time_map[:10])

    #
    si, si_arr = get_si4(rate_map,
                         time_map,
                         verbose)

    #
    t = np.arange(rate_map.shape[0])/30-10
    plt.plot(t, rate_map
                #c=clrs[cell_id]
                )
    
    #
    plt.title(str(cell_id)+ ", Si: "+str(np.round(si,2)))

    # plot vertical line at t = 0
    plt.plot([t[t.shape[0]//2],t[t.shape[0]//2]],[0,np.max(rate_map)],'r--')

    #
    ctr+=1

    #
    if ctr>20:
        plt.xlabel("Time (sec)")
    else:
        plt.xticks([])


    if ctr==25:
        break

    #
    if False:
        break

#
plt.suptitle("# of rewards = "+str(reward_times.shape[0]) + ", bin width = "+str(bin_width))

plt.show()


In [ ]:
duration = np.ma.sum(time_map)

print ("duration: ", duration)

In [ ]:
22 + np.spacing(1)

In [ ]:
print (si)
print (si_array)

In [ ]:
##############################################################
######## COMPUTE REWARD TRIGGERED RESPONSE OF ROIs ###########
##############################################################
# 
from utils_calcium import plot_psth_rois_only

#
def init_fig():
    plt.figure(figsize=(20,12))
    axes = []
    for k in range(4):
        ax=plt.subplot(2,2,k+1)
        axes.append(ax) 

    return axes

# 
window = 300
start_cell=0
show_random = True
idx_cells = None
smoothing = True
plot_separately = False
c.plot_rois_only = True
c.cells_to_plot = c.best_matches
c.cells_to_plot = np.random.choice(np.arange(c.sessions[0].F_filtered.shape[0]),
                                   4,
                                   replace=False)
#
print ("c.cells_to_plot: ", c.cells_to_plot)
#
axes = init_fig()

#
for session_id in np.arange(1,9,1):

    #
    psths_avg, psths_shuffled_avg, idx_cells, n_bursts  = get_reward_triggered_psth(session_id, 
                                                                                    c,
                                                                                    window,
                                                                                    idx_cells)
    #
    plot_psth_rois_only(c,
                        psths_avg,
                        psths_shuffled_avg,
                        session_id,
                        axes,
                        start_cell,
                        idx_cells,
                        show_random,
                        smoothing,
                        window
                        )


    # reset axes
    if plot_separately:
        #
        plt.suptitle(c.animal_id+" session: "+str(session_id),fontsize=20)
        #
        plt.savefig('/home/cat/'+str(start_cell)+"_session_"+str(session_id)+'.png',dpi=300)
        plt.close()

        axes = init_fig()

#
plt.suptitle(c.animal_id+" session: "+str(session_id),fontsize=20)
#
plt.savefig('/home/cat/'+str(start_cell)+"_session_"+str(session_id)+'.png',dpi=300)
plt.close()


In [ ]:
##############################################################
######## COMPUTE REWARD TRIGGERED RESPONSE OF CELLS ##########
##############################################################
window = 30*6
psth_array = []
idx_cells = None
global_order = False
session_ids = np.arange(1,len(c.session_ids),1)

#
n_bursts_array = []
for session_id in session_ids:
#for session_id in [1,7]:

    #
    psths_avg, psths_shuffled_avg, idx_cells, n_bursts  = get_reward_triggered_psth(session_id, 
                                                                                    c,
                                                                                    window,
                                                                                    idx_cells)
    
    #
    n_bursts_array.append(n_bursts)
    
    #
    psth_array.append(psths_avg[idx_cells[::-1]])

    if global_order==False:
        idx_cells = None

#
psth_array = np.array(psth_array)
print (psth_array.shape)

#
vmax = 0.3
sua = plot_multi_session_psth_imshow(c,
                                session_ids,
                                psth_array,
                                window,
                                n_bursts_array,
                                vmax=vmax)


In [ ]:
#######################################
#######################################
#######################################

# binarize the ensembel state    
c.plotting = False
c.scale_threshold = 0.05 #                 # we want to decrease the burst detection threshold manually - because the 
                                            # automatic threshold detection is too conservative 
c.dff_threshold = 0.1
c.percentile_threshold = 0.999

#
#print ("TODO: need to add the standaline binairzation code to the binarize2pcalcium package....")
#print (" This is generally a good idea anyways for doing single time sieres extraction")

# right now only binarizing upphase - if want to do negative phase have to invert the signal
c.sample_rate = 30
c.binarize_ensemble_state_multi()


In [ ]:
#######################################################
#######################################################
#######################################################

plt.figure()
t = np.arange(c.sessions[1].e_state.shape[0])/30.

# make viridis colormap for line plotting up to 7 sessions
cmap = matplotlib.cm.get_cmap('viridis')
clrs = cmap(np.linspace(0, 1, 7))

#
for session_id in range(1,8,1):
    temp = c.sessions[session_id].e_state_bin 
    temp2 = c.sessions[session_id].e_state

    plt.plot(t, temp + 2*session_id, 
             label = "bmi "+str(session_id),
             c='black',
             alpha=.5)
    
    plt.plot(t, temp2 + 2*session_id, 
             c=clrs[session_id-1])

#
plt.xlabel("Time (sec)",fontsize=16)

#
plt.xlim(0, t[-1])
plt.yticks([])

#c.sessions[1]
plt.legend()

#
plt.suptitle("Ensemble state + Binarization", fontsize=16)

plt.show()


In [ ]:
#######################################
#######################################
#######################################

#
#c.sessions[session_id].thresh = thresh
c.overwrite_spreadsheet = True
c.plotting = True
c.update_spreadsheet = True
c.burst_sign = 1         # -1 or +1
c.process_snippets()           # processes snippets and adds them to the spreassheet 

print ("TO ADD: grab the tones also during the bursts")



In [ ]:
###############################################
###############################################
###############################################

#
#c.time_warp = False
c.plotting = True

#c.use_rewarded_bursts_only = False
c.get_mean_warped_trace()  # this computes the warp trace regardless

In [ ]:
################################################
######### VISUALIZE WARPING FUNCITONS ##########
################################################

#
c.use_stretched = True
c.plotting = True
c.burst_to_threshold_only = True                 # this uses the bursts only to the threshold
c.compute_warping_functions()


In [ ]:
# this plots the best cell responses to bursts
c.start_idx = 0
c.pre_burst_n_time_pts = 200
c.ca_type = 'DFF'

# 
c.plot_warped_cell_ca_trial_and_burst()

#

In [ ]:
################################################
######### VISUALIZE WARPING FUNCITONS ##########
################################################
# this plots the best cell responses to bursts
c.start_idx = 0
c.plot_warped_cell_ca_burst_only()

#
print ("SOME TODOS")
print (" 1. Warp only to the threshold - not the peak of burst")
print (" 2. compute the rate map for each cell for burst initiation to threshold/end of peak (2 options)")
print (" 3. compute the rate map for each cell from trial beginning to threshold/end of peak (2 options)")


In [ ]:
d = np.load('/media/cat/8TB/donato/bmi/DON-015801/20230612/plane0/merged/binarized_traces.npz', allow_pickle=True)





In [ ]:
F = np.load('/media/cat/8TB/donato/bmi/DON-015801/20230611/plane0/merged/F.npy', allow_pickle=True)
print (F.shape)

#
plt.figure()

#
for k in range(20):
    plt.plot(F[k])

#
plt.show()